In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.models as models
import cv2
import sys
import math
import random
import splitfolders
import torchsummary
from tqdm.auto import tqdm
from ResNet_18 import resnet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
train_df = pd.read_csv('./train_data_action.csv')
test_df = pd.read_csv('./test_data_action.csv')

In [ ]:
le = LabelEncoder()
le = le.fit(train_df['action'])
train_df['action'] = le.transform(train_df['action'])
test_df['action'] = le.transform(test_df['action'])

In [ ]:
path_list = train_df[train_df['action']==14]['img_path']
pl = list(path_list)

In [ ]:
tmp = random.sample(pl, 500)

In [ ]:
img_path_list = []

In [ ]:
for i in range(15):
    path_list = list(train_df[train_df['action']==i]['img_path'])
    if len(path_list) >= 500:
        tmp = random.sample(path_list, 500)
        for i in tmp:
            img_path_list.append(i)
    else:
        for i in path_list:
            img_path_list.append(i)

In [ ]:
df = pd.DataFrame(img_path_list)
df.columns = ['img_path']
df

In [ ]:
path_label_df = pd.merge(train_df, df, on='img_path', how='inner')
path_label_df['user'] = ''
path_label_df

In [ ]:
for i in range(len(path_label_df)):
    if 'user01' in path_label_df.loc[i]['img_path']:
        path_label_df.loc[i, 'user'] = 'user01'
    if 'user02' in path_label_df.loc[i]['img_path']:
        path_label_df.loc[i, 'user'] = 'user02'
    if 'user03' in path_label_df.loc[i]['img_path']:
        path_label_df.loc[i, 'user'] = 'user03'
    if 'user04' in path_label_df.loc[i]['img_path']:
        path_label_df.loc[i, 'user'] = 'user04'

In [ ]:
path_label_df

In [ ]:
data_path = "./userdata/user01-06/"
dir_list = os.listdir(data_path)

e4eda_all_data = []
dir_list.remove('user05')
dir_list.remove('user06')

# e4Eda_df = pd.DataFrame([], columns=['ts','e4Eda_path'])

for user_data in dir_list:
    sub_list = os.listdir(data_path + user_data)
    for item in sub_list:
        e4_path = data_path + user_data + "/" + item + "/e4Eda"
        e4_list = os.listdir(e4_path)
        for e4_data in e4_list:
            a = float(e4_data[:-4])
            e4eda_all_data.append([a, e4_path + "/" + e4_data])
        
e4Eda_df = pd.DataFrame(e4eda_all_data, columns=['ts','e4Eda_path'])
e4Eda_df['user'] = ''
e4Eda_df

In [ ]:
for i in range(len(e4Eda_df)):
    if 'user01' in e4Eda_df.loc[i]['e4Eda_path']:
        e4Eda_df.loc[i, 'user'] = 'user01'
    if 'user02' in e4Eda_df.loc[i]['e4Eda_path']:
        e4Eda_df.loc[i, 'user'] = 'user02'
    if 'user03' in e4Eda_df.loc[i]['e4Eda_path']:
        e4Eda_df.loc[i, 'user'] = 'user03'
    if 'user04' in e4Eda_df.loc[i]['e4Eda_path']:
        e4Eda_df.loc[i, 'user'] = 'user04'

In [ ]:
e4Eda_df

In [ ]:
new_df = pd.merge(path_label_df, e4Eda_df, on = 'ts', how = 'inner')
new_df

In [ ]:
for i in range(len(new_df)):
    if new_df.loc[i, 'user_x'] != new_df.loc[i, 'user_y']:
        new_df.drop([i], axis=0, inplace=True)

In [ ]:
new_df = new_df.drop(['user_x'], axis='columns')

In [ ]:
new_df.rename(columns={'user_y': 'user'}, inplace=True)
new_df

In [ ]:
new_df.to_csv('eda_img_label.csv')

In [ ]:
test_df
test_df['user'] = ''
test_df

In [ ]:
for i in range(len(test_df)):
    if 'user01' in test_df.loc[i]['img_path']:
        test_df.loc[i, 'user'] = 'user01'
    if 'user02' in test_df.loc[i]['img_path']:
        test_df.loc[i, 'user'] = 'user02'
    if 'user03' in test_df.loc[i]['img_path']:
        test_df.loc[i, 'user'] = 'user03'
    if 'user04' in test_df.loc[i]['img_path']:
        test_df.loc[i, 'user'] = 'user04'

In [ ]:
test_df

In [ ]:
e4Eda_df

In [ ]:
test_new_df = pd.merge(test_df, e4Eda_df, on = 'ts', how = 'inner')
test_new_df

In [ ]:
for i in range(len(new_df)):
    if test_new_df.loc[i, 'user_x'] != test_new_df.loc[i, 'user_y']:
        test_new_df.drop([i], axis=0, inplace=True)

In [ ]:
test_new_df.to_csv('eda_img_label_test.csv')

밑으로 LSTM training을 위한 코드

In [16]:
for i in range(len(train_df)):
    csv_path = train_df.loc[i, 'e4Eda_path']
    tmp_df = pd.read_csv(csv_path)
    mean = tmp_df['eda'].mean()
    train_df.loc[i, 'mean_e4'] = mean
train_df
train_df.to_csv('train_label_data.csv')

In [4]:
train_df['action'].value_counts()

14    498
11    498
5     493
3     489
13    476
4     476
6     476
7     468
10    467
8     459
12    438
2     341
9     141
1       2
Name: action, dtype: int64

In [5]:
test_df['action'].value_counts()

14    3329
13    1193
8     1121
4      975
3      568
7      498
11     479
5      426
6      303
10     133
12     109
2       84
9       37
1        2
Name: action, dtype: int64

In [6]:
le = LabelEncoder()
le = le.fit(train_df['action'])
train_df['action'] = le.transform(train_df['action'])
test_df['action'] = le.transform(test_df['action'])

In [8]:
train_df['mean_e4'] = ''
train_df

,Unnamed: 0.1,Unnamed: 0,ts,action,img_path,e4Eda_path,user,mean_e4
0,0,29029,1.598920e+09,13,./train/data/user04/1598912400/RP/1598919840.png,./userdata/user01-06/user04/1598912400/e4Eda/1...,user04,
1,2,28976,1.598955e+09,2,./train/data/user04/1598912400/RP/1598954760.png,./userdata/user01-06/user04/1598912400/e4Eda/1...,user04,
2,3,35,1.598878e+09,11,./train/data/user01/1598832660/RP/1598878440.png,./userdata/user01-06/user01/1598832660/e4Eda/1...,user01,
3,7,28538,1.598846e+09,12,./train/data/user04/1598828400/RP/1598846460.png,./userdata/user01-06/user04/1598828400/e4Eda/1...,user04,
4,9,10644,1.599904e+09,9,./train/data/user02/1599893100/RP/1599903840.png,./userdata/user01-06/user02/1599893100/e4Eda/1...,user02,
...,...,...,...,...,...,...,...,...
5717,13969,10794,1.599976e+09,7,./train/data/user02/1599922800/RP/1599976080.png,./userdata/user01-06/user02/1599922800/e4Eda/1...,user02,
5718,13973,22327,1.600524e+09,7,./train/data/user03/1600448400/RP/1600523700.png,./userdata/user01-06/user03/1600448400/e4Eda/1...,user03,
5719,13976,18809,1.600076e+09,5,./train/data/user03/1600020000/RP/1600076460.png,./userdata/user01-06/user03/1600020000/e4Eda/1...,user03,
5720,13977,13877,1.601099e+09,5,./train/data/user02/1601046000/RP/1601098980.png,./userdata/user01-06/user02/1601046000/e4Eda/1...,user02,


In [12]:
for i in range(len(test_df)):
    csv_path = test_df.loc[i, 'e4Eda_path']
    tmp_df = pd.read_csv(csv_path)
    mean = tmp_df['eda'].mean()
    test_df.loc[i, 'mean_e4'] = mean
test_df

,Unnamed: 0.1,Unnamed: 0,ts,action,img_path,e4Eda_path,user,mean_e4
0,3,5482,1.600573e+09,12,./train/data/user01/1600527600/RP/1600572600.png,./userdata/user01-06/user01/1600527600/e4Eda/1...,user01,6.102752
1,5,46781,1.601090e+09,7,./train/data/user04/1601079300/RP/1601089620.png,./userdata/user01-06/user04/1601079300/e4Eda/1...,user04,3.043166
2,8,41251,1.600476e+09,7,./train/data/user04/1600470000/RP/1600475940.png,./userdata/user01-06/user04/1600470000/e4Eda/1...,user04,0.023352
3,9,32495,1.599353e+09,6,./train/data/user04/1599347400/RP/1599352560.png,./userdata/user01-06/user04/1599347400/e4Eda/1...,user04,3.622869
4,10,34492,1.599566e+09,4,./train/data/user04/1599522960/RP/1599566280.png,./userdata/user01-06/user04/1599522960/e4Eda/1...,user04,28.459895
...,...,...,...,...,...,...,...,...
9252,21560,31755,1.599278e+09,3,./train/data/user04/1599261300/RP/1599278160.png,./userdata/user01-06/user04/1599261300/e4Eda/1...,user04,7.839923
9253,21562,35250,1.599729e+09,13,./train/data/user04/1599690000/RP/1599728880.png,./userdata/user01-06/user04/1599690000/e4Eda/1...,user04,12.145136
9254,21565,15701,1.599102e+09,3,./train/data/user03/1599058980/RP/1599101940.png,./userdata/user01-06/user03/1599058980/e4Eda/1...,user03,2.129693
9255,21566,10536,1.599898e+09,3,./train/data/user02/1599893100/RP/1599897600.png,./userdata/user01-06/user02/1599893100/e4Eda/1...,user02,0.290836


In [20]:
train_df = pd.read_csv('./train_label_data.csv')
test_df = pd.read_csv('./test_label_data.csv')

In [21]:
test_df.to_csv('test_label_data.csv')

In [22]:
train_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ts,action,img_path,e4Eda_path,user,mean_e4
0,0,0,29029,1.598920e+09,13,./train/data/user04/1598912400/RP/1598919840.png,./userdata/user01-06/user04/1598912400/e4Eda/1...,user04,15.204676
1,1,2,28976,1.598955e+09,2,./train/data/user04/1598912400/RP/1598954760.png,./userdata/user01-06/user04/1598912400/e4Eda/1...,user04,10.504916
2,2,3,35,1.598878e+09,11,./train/data/user01/1598832660/RP/1598878440.png,./userdata/user01-06/user01/1598832660/e4Eda/1...,user01,38.750507
3,3,7,28538,1.598846e+09,12,./train/data/user04/1598828400/RP/1598846460.png,./userdata/user01-06/user04/1598828400/e4Eda/1...,user04,8.082568
4,4,9,10644,1.599904e+09,9,./train/data/user02/1599893100/RP/1599903840.png,./userdata/user01-06/user02/1599893100/e4Eda/1...,user02,0.209014
...,...,...,...,...,...,...,...,...,...
5717,5717,13969,10794,1.599976e+09,7,./train/data/user02/1599922800/RP/1599976080.png,./userdata/user01-06/user02/1599922800/e4Eda/1...,user02,0.056583
5718,5718,13973,22327,1.600524e+09,7,./train/data/user03/1600448400/RP/1600523700.png,./userdata/user01-06/user03/1600448400/e4Eda/1...,user03,0.108481
5719,5719,13976,18809,1.600076e+09,5,./train/data/user03/1600020000/RP/1600076460.png,./userdata/user01-06/user03/1600020000/e4Eda/1...,user03,0.002386
5720,5720,13977,13877,1.601099e+09,5,./train/data/user02/1601046000/RP/1601098980.png,./userdata/user01-06/user02/1601046000/e4Eda/1...,user02,0.006131


In [23]:
test_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ts,action,img_path,e4Eda_path,user,mean_e4
0,0,3,5482,1.600573e+09,12,./train/data/user01/1600527600/RP/1600572600.png,./userdata/user01-06/user01/1600527600/e4Eda/1...,user01,6.102752
1,1,5,46781,1.601090e+09,7,./train/data/user04/1601079300/RP/1601089620.png,./userdata/user01-06/user04/1601079300/e4Eda/1...,user04,3.043166
2,2,8,41251,1.600476e+09,7,./train/data/user04/1600470000/RP/1600475940.png,./userdata/user01-06/user04/1600470000/e4Eda/1...,user04,0.023352
3,3,9,32495,1.599353e+09,6,./train/data/user04/1599347400/RP/1599352560.png,./userdata/user01-06/user04/1599347400/e4Eda/1...,user04,3.622869
4,4,10,34492,1.599566e+09,4,./train/data/user04/1599522960/RP/1599566280.png,./userdata/user01-06/user04/1599522960/e4Eda/1...,user04,28.459895
...,...,...,...,...,...,...,...,...,...
9252,9252,21560,31755,1.599278e+09,3,./train/data/user04/1599261300/RP/1599278160.png,./userdata/user01-06/user04/1599261300/e4Eda/1...,user04,7.839923
9253,9253,21562,35250,1.599729e+09,13,./train/data/user04/1599690000/RP/1599728880.png,./userdata/user01-06/user04/1599690000/e4Eda/1...,user04,12.145136
9254,9254,21565,15701,1.599102e+09,3,./train/data/user03/1599058980/RP/1599101940.png,./userdata/user01-06/user03/1599058980/e4Eda/1...,user03,2.129693
9255,9255,21566,10536,1.599898e+09,3,./train/data/user02/1599893100/RP/1599897600.png,./userdata/user01-06/user02/1599893100/e4Eda/1...,user02,0.290836


In [24]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [25]:
CFG = {
    'EPOCHS':50,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':42
}

In [26]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [261]:
train, val, _, _ = train_test_split(train_df, train_df['action'], test_size=0.1, random_state=CFG['SEED'])

In [262]:
train

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ts,action,img_path,e4Eda_path,user,mean_e4
5333,5333,13065,32728,1.599355e+09,2,./train/data/user04/1599347400/RP/1599354720.png,./userdata/user01-06/user04/1599347400/e4Eda/1...,user04,4.932050
245,245,594,37278,1.599998e+09,10,./train/data/user04/1599957060/RP/1599997860.png,./userdata/user01-06/user04/1599957060/e4Eda/1...,user04,4.167552
2805,2805,6994,17045,1.599470e+09,2,./train/data/user03/1599422400/RP/1599470340.png,./userdata/user01-06/user03/1599422400/e4Eda/1...,user03,0.303244
367,367,900,40689,1.600432e+09,2,./train/data/user04/1600380900/RP/1600431660.png,./userdata/user01-06/user04/1600380900/e4Eda/1...,user04,4.706330
2342,2342,5871,22645,1.600485e+09,2,./train/data/user03/1600448400/RP/1600485120.png,./userdata/user01-06/user03/1600448400/e4Eda/1...,user03,3.299797
...,...,...,...,...,...,...,...,...,...
3772,3772,9302,33445,1.599451e+09,13,./train/data/user04/1599430860/RP/1599451320.png,./userdata/user01-06/user04/1599430860/e4Eda/1...,user04,5.231229
5191,5191,12734,8255,1.601192e+09,9,./train/data/user01/1601166300/RP/1601191860.png,./userdata/user01-06/user01/1601166300/e4Eda/1...,user01,0.988020
5226,5226,12809,11135,1.599967e+09,11,./train/data/user02/1599922800/RP/1599967380.png,./userdata/user01-06/user02/1599922800/e4Eda/1...,user02,0.077779
5390,5390,13195,41238,1.600488e+09,10,./train/data/user04/1600470000/RP/1600488240.png,./userdata/user01-06/user04/1600470000/e4Eda/1...,user04,2.476982


In [263]:
train.drop(labels=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'],axis=1)
val.drop(labels=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'],axis=1)

,ts,action,img_path,e4Eda_path,user,mean_e4
1600,1.600095e+09,1,./train/data/user03/1600020000/RP/1600095060.png,./userdata/user01-06/user03/1600020000/e4Eda/1...,user03,0.237270
1121,1.599119e+09,13,./train/data/user04/1599085500/RP/1599118620.png,./userdata/user01-06/user04/1599085500/e4Eda/1...,user04,9.783710
4328,1.599931e+09,12,./train/data/user01/1599922800/RP/1599931080.png,./userdata/user01-06/user01/1599922800/e4Eda/1...,user01,2.343516
3827,1.601208e+09,4,./train/data/user03/1601141400/RP/1601207640.png,./userdata/user01-06/user03/1601141400/e4Eda/1...,user03,2.077572
2131,1.601039e+09,6,./train/data/user03/1600963200/RP/1601039100.png,./userdata/user01-06/user03/1600963200/e4Eda/1...,user03,0.236553
...,...,...,...,...,...,...
1536,1.601109e+09,10,./train/data/user04/1601079300/RP/1601109300.png,./userdata/user01-06/user04/1601079300/e4Eda/1...,user04,4.021606
4841,1.600330e+09,4,./train/data/user03/1600286400/RP/1600330020.png,./userdata/user01-06/user03/1600286400/e4Eda/1...,user03,2.490548
468,1.599991e+09,3,./train/data/user01/1599922800/RP/1599991200.png,./userdata/user01-06/user01/1599922800/e4Eda/1...,user01,2.443359
4065,1.601131e+09,7,./train/data/user03/1601054580/RP/1601130720.png,./userdata/user01-06/user03/1601054580/e4Eda/1...,user03,0.159062
